In [5]:
import vitaldb
import numpy as np
import pandas as pd
import scipy.signal as ss
from scipy.ndimage import minimum_filter1d, maximum_filter1d
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm.auto import tqdm
import random

# colorblind friendly palette
import matplotlib as mpl
cp = ["#172A5A", "#FF7171", "#227567", "#34BAEA", "#F9D466", ]
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=cp)

# set default font
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Open Sans', 'Arial']
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
def um(ta=None, tb=None, data=[[], [], []], t=[]):
    ta = ta if ta else t.min()
    tb = tb if tb else t.max()
    twin = (t >= ta) & (t <= tb)

    plt.figure(figsize=(12,8), dpi=100)
    for i, sig in enumerate(data):
        sig = np.ma.masked_invalid(sig)
        plt.subplot(3,1,i+1)
        plt.plot(t[twin], sig[twin], lw=1, c=cp[i])
        sns.despine(bottom=i<2)
        if i<2: plt.xticks([])
        plt.xlim(ta, tb)
        plt.tight_layout()
        plt.show()

In [3]:
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information

In [4]:
caseids = list(
    set(df_trks[df_trks['tname'] == 'SNUADC/ART']['caseid']) &
    set(df_trks[df_trks['tname'] == 'SNUADC/ECG_II']['caseid']) &
    set(df_trks[df_trks['tname'] == 'SNUADC/PLETH']['caseid']) &
    set(df_cases[df_cases['age'] > 18]['caseid']) &
    set(df_cases[df_cases['age'] >= 18]['caseid']) & 
    set(df_cases[df_cases['weight'] >= 30]['caseid']) & 
    set(df_cases[df_cases['weight'] < 140]['caseid']) & 
    set(df_cases[df_cases['height'] >= 135]['caseid']) & 
    set(df_cases[df_cases['height'] < 200]['caseid']) & 
    set(df_cases[~df_cases['opname'].str.contains("transplant", case=False)]['caseid']) & 
    set(df_cases[~df_cases['opname'].str.contains("aneurysm", case=False)]['caseid']) & 
    set(df_cases[~df_cases['opname'].str.contains("aorto", case=False)]['caseid'])& 
    set(df_cases[df_cases['ane_type'] == 'General']['caseid'])
)
print('Total {} cases found'.format(len(caseids)))

Total 3235 cases found


In [14]:
random_inds = random.sample(range(3235), 10)
fs = 125

for ind in random_inds:
    track_names = ['ECG_II', 'ART', 'PLETH']
    vf = vitaldb.VitalFile(caseids[ind], track_names) 
    data = vf.to_numpy(track_names, 1/fs)
    t = np.arange(data.shape[0]) / fs
    
    ecg, art, ppg = data[:, 0], data[:, 1], data[:, 2]
    
    fourier_coeffs_art = np.fft.fft(art)
    fourier_coeffs_ppg = np.fft.fft(ppg)
    
    print(f'Number of ART NaNs: {np.count_nonzero(np.isnan(art))}', end = "\n")
    print(f'Number of PPG NaNs: {np.count_nonzero(np.isnan(ppg))}', end = "\n")
    print(f'Percent NaN: {np.count_nonzero(np.isnan(ppg)) / len(np.isnan(ppg))}', end = "\n")
    print(f'Length: {len(np.isnan(ppg))}', end = "\n")   

    
    # plt.figure(figsize=(8,4))
    # plt.subplot(2,1,1)
    # plt.plot(t, art, lw=1, label = "Original ART")
    # #plt.xlim([0, len(ppg)])
    # sns.despine()
    # plt.gca().margins(0)
    # plt.legend()
    # plt.subplot(2, 1, 2)
    # plt.plot(np.abs(fourier_coeffs_art), label='real part')
    # plt.legend()
    # plt.show()
    
    # plt.figure(figsize=(8,4))
    # plt.subplot(2,1,1)
    # plt.plot(t, ppg, lw=1, label = "Original PPG")
    # #plt.xlim([0, len(ppg)])
    # sns.despine()
    # plt.gca().margins(0)
    # plt.legend()
    # plt.subplot(2, 1, 2)
    # plt.plot(np.abs(fourier_coeffs_ppg), label='real part')
    # plt.legend()
    # plt.show()

Number of ART NaNs: 112
Number of PPG NaNs: 112
Percent NaN: 5.726920001063571e-05
Length: 1955676
Number of ART NaNs: 194
Number of PPG NaNs: 194
Percent NaN: 0.0002007570818612872
Length: 966342
Number of ART NaNs: 319
Number of PPG NaNs: 319
Percent NaN: 0.00020988733852985434
Length: 1519863
Number of ART NaNs: 97
Number of PPG NaNs: 97
Percent NaN: 4.84421600608873e-05
Length: 2002388
Number of ART NaNs: 21032
Number of PPG NaNs: 21032
Percent NaN: 0.005493516673444755
Length: 3828513
Number of ART NaNs: 4125
Number of PPG NaNs: 4125
Percent NaN: 0.003396017644472765
Length: 1214658
Number of ART NaNs: 2418
Number of PPG NaNs: 2418
Percent NaN: 0.0015255876635606005
Length: 1584963
Number of ART NaNs: 2535
Number of PPG NaNs: 2535
Percent NaN: 0.0013070122884934453
Length: 1939538
Number of ART NaNs: 3438
Number of PPG NaNs: 3438
Percent NaN: 0.006068265337461213
Length: 566554
Number of ART NaNs: 2902
Number of PPG NaNs: 2902
Percent NaN: 0.0017848027307112765
Length: 1625950
